In [223]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [224]:
path_2020 = "../data/raw/2020_season/nba2021_per_game.csv"
path_2021 = "../data/raw/2021_season/2021-2022 NBA Player Stats - Regular.csv"
path_2022 = "../data/raw/2022_season/2022-2023 NBA Player Stats - Regular.csv"

In [225]:
df_2022 = pd.read_csv(path_2022, encoding="latin-1", sep=";")
df_2021 = pd.read_csv(path_2021, sep=";", encoding="latin-1")

# Visualización de los Dataframes


## Temporada 2021 - 2022


In [226]:
df_2021.head(20)

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Precious Achiuwa,C,22,TOR,73,28,23.6,3.6,8.3,...,0.595,2.0,4.5,6.5,1.1,0.5,0.6,1.2,2.1,9.1
1,2,Steven Adams,C,28,MEM,76,75,26.3,2.8,5.1,...,0.543,4.6,5.4,10.0,3.4,0.9,0.8,1.5,2.0,6.9
2,3,Bam Adebayo,C,24,MIA,56,56,32.6,7.3,13.0,...,0.753,2.4,7.6,10.1,3.4,1.4,0.8,2.6,3.1,19.1
3,4,Santi Aldama,PF,21,MEM,32,0,11.3,1.7,4.1,...,0.625,1.0,1.7,2.7,0.7,0.2,0.3,0.5,1.1,4.1
4,5,LaMarcus Aldridge,C,36,BRK,47,12,22.3,5.4,9.7,...,0.873,1.6,3.9,5.5,0.9,0.3,1.0,0.9,1.7,12.9
5,6,Nickeil Alexander-Walker,SG,23,TOT,65,21,22.6,3.9,10.5,...,0.743,0.6,2.3,2.9,2.4,0.7,0.4,1.4,1.6,10.6
6,6,Nickeil Alexander-Walker,SG,23,NOP,50,19,26.3,4.7,12.6,...,0.722,0.7,2.6,3.3,2.8,0.8,0.4,1.7,1.8,12.8
7,6,Nickeil Alexander-Walker,SG,23,UTA,15,2,9.9,1.1,3.2,...,0.917,0.1,1.5,1.5,1.1,0.3,0.3,0.5,1.0,3.5
8,7,Grayson Allen,SG,26,MIL,66,61,27.3,3.9,8.6,...,0.865,0.5,2.9,3.4,1.5,0.7,0.3,0.7,1.5,11.1
9,8,Jarrett Allen,C,23,CLE,56,56,32.3,6.6,9.7,...,0.708,3.4,7.3,10.8,1.6,0.8,1.3,1.7,1.7,16.1


## Temporada 2022 - 2023

In [227]:
df_2022

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Precious Achiuwa,C,23,TOR,55,12,20.7,3.6,7.3,...,0.702,1.8,4.1,6.0,0.9,0.6,0.5,1.1,1.9,9.2
1,2,Steven Adams,C,29,MEM,42,42,27.0,3.7,6.3,...,0.364,5.1,6.5,11.5,2.3,0.9,1.1,1.9,2.3,8.6
2,3,Bam Adebayo,C,25,MIA,75,75,34.6,8.0,14.9,...,0.806,2.5,6.7,9.2,3.2,1.2,0.8,2.5,2.8,20.4
3,4,Ochai Agbaji,SG,22,UTA,59,22,20.5,2.8,6.5,...,0.812,0.7,1.3,2.1,1.1,0.3,0.3,0.7,1.7,7.9
4,5,Santi Aldama,PF,22,MEM,77,20,21.8,3.2,6.8,...,0.750,1.1,3.7,4.8,1.3,0.6,0.6,0.8,1.9,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
674,535,Thaddeus Young,PF,34,TOR,54,9,14.7,2.0,3.7,...,0.692,1.3,1.8,3.1,1.4,1.0,0.1,0.8,1.6,4.4
675,536,Trae Young,PG,24,ATL,73,73,34.8,8.2,19.0,...,0.886,0.8,2.2,3.0,10.2,1.1,0.1,4.1,1.4,26.2
676,537,Omer Yurtseven,C,24,MIA,9,0,9.2,1.8,3.0,...,0.833,0.9,1.7,2.6,0.2,0.2,0.2,0.4,1.8,4.4
677,538,Cody Zeller,C,30,MIA,15,2,14.5,2.5,3.9,...,0.686,1.7,2.6,4.3,0.7,0.2,0.3,0.9,2.2,6.5


# Limpieza de datos

## Repetición de jugadores

In [228]:
# Dentro de los datasets, podemos ver que hay ciertos jugadores que se repiten en varias ocaciones, esto es debido a que
# durante la temporada cambiaron de equipo, para proseguir tenemos que quitar a los jugadores duplicados y solo dejar 
# la fila que contiene sus promedios generales. Además ya que la fila de los promedios generales, en la sección de equipo
# solo dice "TOT" tendremos que cambiar el "TOT" por el último equipo en el que jugó el jugador

In [229]:
def eliminador(df): #Iniciamos una función que va a eliminar los valores duplicados
    idx = []
    ultimo_equipo = {}
    contador = 0
    for id, player in enumerate(df["Player"]): #vamos iterando a traves de los jugadores y de su índice
        if player == df["Player"].iloc[id - 1]: #si el jugador es igual al pasado guardamos su índice en la lista idx
            contador +=1
            if contador == 2:
                ultimo_equipo[player] =  df["Tm"].iloc[id] #La segunda vez que se repite un jugador guardamos el jugador y su equipo en ultimo_equipo
                contador = 0
            idx.append(id)
        else:
            contador = 0
    df.drop(idx, inplace=True) #Tiramos las filas repetidas

    for jugador, equipo in ultimo_equipo.items():
        idx_tot = df[(df["Player"] == jugador) & (df["Tm"] == "TOT")].index #sacamos el índice de la fila de promedios del jugador repetido
        df.loc[idx_tot, "Tm"] = equipo  #Sustituímos el valor "TOT" por el último equipo en dónde estuvo el jugador
    return df

## Jugadores ausentes

In [230]:
# Ya que existe la posibilidad de que existan jugadores que su último temporada fue en 2021 o que su primera temporada fue en
# 2022, voy a tomar la decisión de eliminar a esos jugadores y solo quedarme con loq ue estuvieron activos ambos años.

In [231]:
def ausente(df1, df2):
    for player in df1["Player"]:
        if player not in df2["Player"].values: #Revisamos si el jugador se encuentra en el año siguiente o pasado
            df1.drop(df1[df1["Player"] == player].index, inplace=True) #En caso de que no se encuentre, tiraremos esa fila
    return df1

## Ajuste de índice

In [232]:
# El valor que tienen en común ambos datsets son los nombres de los jugadores, entonces para poder concatenar los df de manera
# exitosa, haremos que la columna "Players" se convierta en el índice. Además tiraremos la columna "Rk", ya que ya no sirve
# ninguna función.

In [233]:
def continuo(df):
    df = df.set_index(['Player']) #Cambiamos "Player" a ser el nuevo índice
    df = df.drop(["Rk"], axis=1) # Tiramos la columna "Rk"
    return df

## Cambio de nombre columnas

In [234]:
# Ahora cada columna del dataframe le cambiaremos el nombre, esto con el fin de poder identificar las estadísticas del 2021 y las
# del 2022 al momento de concatenar los dataframes.

In [235]:
def nombre_col(df, nuevo_nombre):
    columnas = [col for col in df.columns] # Sacamos una lista de todas las columnas
    columnas_nuevas = {col: col + " " + nuevo_nombre for col in columnas} #Hacemos un diccionario con el nombre de la columna y su nuevo nombre
    df = df.rename(columns = columnas_nuevas) # Apliamos los cambios en las columnas
    return df

## Aplicación de los cambios

In [236]:
df_2021 = ausente(eliminador(df_2021), df_2022)
df_2022 = ausente(eliminador(df_2022), df_2021)
df_2021 = nombre_col(continuo(df_2021), "2021")
df_2022 = nombre_col(continuo(df_2022), "2022")


## Concatenación

In [243]:
df_comb = df_2021.join(df_2022)
df_comb

,Pos 2021,Age 2021,Tm 2021,G 2021,GS 2021,MP 2021,FG 2021,FGA 2021,FG% 2021,3P 2021,...,FT% 2022,ORB 2022,DRB 2022,TRB 2022,AST 2022,STL 2022,BLK 2022,TOV 2022,PF 2022,PTS 2022
Player,,,,,,,,,,,,,,,,,,,,,
Precious Achiuwa,C,22,TOR,73,28,23.6,3.6,8.3,0.439,0.8,...,0.702,1.8,4.1,6.0,0.9,0.6,0.5,1.1,1.9,9.2
Steven Adams,C,28,MEM,76,75,26.3,2.8,5.1,0.547,0.0,...,0.364,5.1,6.5,11.5,2.3,0.9,1.1,1.9,2.3,8.6
Bam Adebayo,C,24,MIA,56,56,32.6,7.3,13.0,0.557,0.0,...,0.806,2.5,6.7,9.2,3.2,1.2,0.8,2.5,2.8,20.4
Santi Aldama,PF,21,MEM,32,0,11.3,1.7,4.1,0.402,0.2,...,0.750,1.1,3.7,4.8,1.3,0.6,0.6,0.8,1.9,9.0
Nickeil Alexander-Walker,SG,23,UTA,65,21,22.6,3.9,10.5,0.372,1.6,...,0.667,0.3,1.5,1.7,1.8,0.5,0.4,0.9,1.5,6.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Thaddeus Young,PF,33,TOR,52,1,16.3,2.7,5.2,0.518,0.3,...,0.692,1.3,1.8,3.1,1.4,1.0,0.1,0.8,1.6,4.4
Trae Young,PG,23,ATL,76,76,34.9,9.4,20.3,0.460,3.1,...,0.886,0.8,2.2,3.0,10.2,1.1,0.1,4.1,1.4,26.2
Omer Yurtseven,C,23,MIA,56,12,12.6,2.3,4.4,0.526,0.0,...,0.833,0.9,1.7,2.6,0.2,0.2,0.2,0.4,1.8,4.4


In [238]:
df_2021

,Pos 2021,Age 2021,Tm 2021,G 2021,GS 2021,MP 2021,FG 2021,FGA 2021,FG% 2021,3P 2021,...,FT% 2021,ORB 2021,DRB 2021,TRB 2021,AST 2021,STL 2021,BLK 2021,TOV 2021,PF 2021,PTS 2021
Player,,,,,,,,,,,,,,,,,,,,,
Precious Achiuwa,C,22,TOR,73,28,23.6,3.6,8.3,0.439,0.8,...,0.595,2.0,4.5,6.5,1.1,0.5,0.6,1.2,2.1,9.1
Steven Adams,C,28,MEM,76,75,26.3,2.8,5.1,0.547,0.0,...,0.543,4.6,5.4,10.0,3.4,0.9,0.8,1.5,2.0,6.9
Bam Adebayo,C,24,MIA,56,56,32.6,7.3,13.0,0.557,0.0,...,0.753,2.4,7.6,10.1,3.4,1.4,0.8,2.6,3.1,19.1
Santi Aldama,PF,21,MEM,32,0,11.3,1.7,4.1,0.402,0.2,...,0.625,1.0,1.7,2.7,0.7,0.2,0.3,0.5,1.1,4.1
Nickeil Alexander-Walker,SG,23,UTA,65,21,22.6,3.9,10.5,0.372,1.6,...,0.743,0.6,2.3,2.9,2.4,0.7,0.4,1.4,1.6,10.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Thaddeus Young,PF,33,TOR,52,1,16.3,2.7,5.2,0.518,0.3,...,0.469,1.5,2.5,4.0,2.0,1.0,0.3,1.0,1.6,6.2
Trae Young,PG,23,ATL,76,76,34.9,9.4,20.3,0.460,3.1,...,0.904,0.7,3.1,3.7,9.7,0.9,0.1,4.0,1.7,28.4
Omer Yurtseven,C,23,MIA,56,12,12.6,2.3,4.4,0.526,0.0,...,0.623,1.5,3.7,5.3,0.9,0.3,0.4,0.7,1.5,5.3


In [239]:
df_2022

,Pos 2022,Age 2022,Tm 2022,G 2022,GS 2022,MP 2022,FG 2022,FGA 2022,FG% 2022,3P 2022,...,FT% 2022,ORB 2022,DRB 2022,TRB 2022,AST 2022,STL 2022,BLK 2022,TOV 2022,PF 2022,PTS 2022
Player,,,,,,,,,,,,,,,,,,,,,
Precious Achiuwa,C,23,TOR,55,12,20.7,3.6,7.3,0.485,0.5,...,0.702,1.8,4.1,6.0,0.9,0.6,0.5,1.1,1.9,9.2
Steven Adams,C,29,MEM,42,42,27.0,3.7,6.3,0.597,0.0,...,0.364,5.1,6.5,11.5,2.3,0.9,1.1,1.9,2.3,8.6
Bam Adebayo,C,25,MIA,75,75,34.6,8.0,14.9,0.540,0.0,...,0.806,2.5,6.7,9.2,3.2,1.2,0.8,2.5,2.8,20.4
Santi Aldama,PF,22,MEM,77,20,21.8,3.2,6.8,0.470,1.2,...,0.750,1.1,3.7,4.8,1.3,0.6,0.6,0.8,1.9,9.0
Nickeil Alexander-Walker,SG,24,MIN,59,3,15.0,2.2,5.0,0.444,1.0,...,0.667,0.3,1.5,1.7,1.8,0.5,0.4,0.9,1.5,6.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Thaddeus Young,PF,34,TOR,54,9,14.7,2.0,3.7,0.545,0.1,...,0.692,1.3,1.8,3.1,1.4,1.0,0.1,0.8,1.6,4.4
Trae Young,PG,24,ATL,73,73,34.8,8.2,19.0,0.429,2.1,...,0.886,0.8,2.2,3.0,10.2,1.1,0.1,4.1,1.4,26.2
Omer Yurtseven,C,24,MIA,9,0,9.2,1.8,3.0,0.593,0.3,...,0.833,0.9,1.7,2.6,0.2,0.2,0.2,0.4,1.8,4.4
